# NFL Data Bowl Analysis

This notebook will go through the methodology of data collection, analysis, and results of a narrowed-down analysis to only passing plays in the 2018 NFL season. The data is obtained from the NFL's 2021 Big Data Bowl: https://www.kaggle.com/c/nfl-big-data-bowl-2021/

*There is potential to combine the 2020 data bowl data, which contains similar info to 2021 data bowl data except about rushing plays 2017-2019. Combining these sources to produce a similar notebook to the original tendency analysis. Less data, but more information in our columns.*

The main focus of this analysis is to see how offensive / defensive personnel matchups, distance from closest defender to targeted receiver, among others later detailed in the notebook. This is a unique opportunity to utilize tracking / location data of players as well.

Other data bowls for reference:
- https://www.kaggle.com/c/nfl-big-data-bowl-2020: Forecast yardage gained on the run plays
- https://www.kaggle.com/c/nfl-big-data-bowl-2022: Analyze special teams data
- https://github.com/nfl-football-ops/Big-Data-Bowl: Inaugural data bowl from 2019, useful R code on animation of tracking

In [147]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None  # default='warn'

In [148]:
bdb_pass = pd.read_csv('nfl-big-data-bowl/pass-2018.csv')
bdb_rush = pd.read_csv('nfl-big-data-bowl/rush.csv')

# Useful Features Present in Both Datasets

Before cleaning the data, from a cursory examination of our data, the main features I think will be useful in this analysis are the following. We can only use pre-snap features in this analysis, or else it defeats the purpose.

- Type of play (run/pass): TARGET (need to create)
- Drive number (need to create / merge with pbp)
- Offensive / defensive team (`HomeTeamAbbr`, `VisitorTeamAbbr`, `PossessionTeam`, need to create `DefTeam`)
- Quarter of the game (`Quarter`)
- Down number (`Down`)
- Time left in a quarter (`GameClock`, need to format this)
- Yards to gain for a first down (`Distance`)
- Yards to gain for a touchdown (100 minus current yardline position, `YardLine`)
- Current score in the game (model as difference, `HomeScoreBeforePlay - VisitorScoreBeforePlay`)
- Offensive formation (`OffenseFormation`)
- Offensive personnel (`OffensePersonnel`)
- Defenders in the box (`DefendersInTheBox`)
- Defensive personnel (`DefensePersonnel`)
- Week of season (`Week`)

# Useful Features Present Only in Rushing Data

Some features are only included in the rushing play data. However, for 2018 alone, we could match up the games and include further information in the analysis for pass plays as well.

- Stadium type (`StadiumType`)
- Turf or grass (`Turf`)
- Weather in game (`GameWeather`)
- Temperature on game day (`Temperature`)
- Humidity on game day (`Humidity`)
- Wind speed on game day (`WindSpeed`)
- Wind direction (`WindDirection`)

## Next Steps / Preparing Data

- Is it more useful to do an analysis on an imbalanced dataset with rushing 2017-2019 and passing 2018, with only the first subset of features?
- Or would I rather narrow down the rushing plays to only 2018, and use all features (can match up `GameID` and thus find the weather/stadium data by game for the passing plays as well)
- Aside from that, still need to complete the following data cleaning steps regardless of which direction I choose:
    - Merge both datasets into same format
    - Ignore post-snap outcomes (i.e. yardage gained, direction of the run, play results, type of pass dropback)
    - Ignore tracking data (for now) since all of it is post-snap movements
        - Look into if I can see whether a play is in "motion" i.e. the WR shifting from the X spot to the slot
        - I have the "time of snap" data, so there may be some potential?
    - Change the offensive formation to one-hot encoded columns
    - Change the personnel for offense and defense to one-hot encoded columns
        - Or find a unique way to deal with this since there are many combinations, some with few observations
    - Narrow down each play to a single row
    - More TBD..

# Merging 2018 run/pass

First I will try the simpler case, where we have more features but must narrow our scope to only 2018 plays. We can match these games on `GameID` which will remove run plays in 2017 and 2019. The key issue is that the run data contains many rows for the same play, since the tracking data is baked in. The solution is to remove the tracking data for the rush dataset, effectively leaving one row per play, then merging the two datasets on their `GameID`. I will do this below by grouping on `PlayID` and only retaining information on the play as a whole. Maybe I'll go back to individual players and see if there's an avenue to work with tracking data / pre-snap motion later and add it in as my own feature...we'll see.

In [149]:
print('Unique pass game data provided: {}'.format(bdb_pass['gameId'].nunique()))
print('Unique rush game data provided: {}   (includes 2017, 2019)'.format(bdb_rush['GameId'].nunique()))
print('Unique pass play data provided: {}'.format(bdb_pass['gameId'].count()))
print('Unique rush play data provided: {} (includes 2017, 2019)'.format(bdb_rush['PlayId'].nunique()))

Unique pass game data provided: 253
Unique rush game data provided: 688   (includes 2017, 2019)
Unique pass play data provided: 19239
Unique rush play data provided: 31007 (includes 2017, 2019)


In [150]:
## Group by play ID, take first since the data we want is all the same
## Don't care for any tracking / individual player data (yet)
bdb_rush_play = bdb_rush.groupby('PlayId').first()

In [151]:
## List of things to drop
to_drop = ['X','Y','S','A','Dis','Dir','Orientation','NflId','DisplayName',
           'JerseyNumber','Season','NflIdRusher','PlayerHeight','PlayerWeight',
           'PlayerBirthDate','PlayerCollegeName','Position', 'PlayDirection',
           'TimeHandoff', 'TimeSnap', 'Yards', 'Team']
bdb_rush = bdb_rush_play.drop(to_drop, axis=1)
## only 2018 rush plays
bdb_rush_2018 = bdb_rush[bdb_rush['GameId'].astype('str').str[:4] == '2018']

In [153]:
print('Unique pass game data provided: {}'.format(bdb_pass['gameId'].nunique()))
print('Unique rush game data provided: {}     (2018)'.format(bdb_rush_2018['GameId'].nunique()))
print('Unique pass play data provided: {}'.format(bdb_pass['gameId'].count()))
print('Unique rush play data provided: {}   (2018)'.format(bdb_rush_2018['GameId'].count()))

Unique pass game data provided: 253
Unique rush game data provided: 256     (2018)
Unique pass play data provided: 19239
Unique rush play data provided: 11271   (2018)


In [154]:
bdb_rush_2018 = bdb_rush_2018.reset_index(drop=True)

In [229]:
## Align bdb_rush_2018 and bdb_pass_2018
## Once these are aligned, we can create general features for the whole dataset at once
to_drop = ['playId', 'playDescription', 'typeDropback', 'penaltyCodes', 'penaltyJerseyNumbers',
           'epa', 'isDefensivePI', 'passResult','playType']
bdb_pass_2018 = bdb_pass.drop(to_drop, axis=1)

In [237]:
bdb_rush_2018.columns

Index(['GameId', 'YardLine', 'Quarter', 'GameClock', 'PossessionTeam', 'Down',
       'Distance', 'FieldPosition', 'HomeScoreBeforePlay',
       'VisitorScoreBeforePlay', 'OffenseFormation', 'OffensePersonnel',
       'DefendersInTheBox', 'DefensePersonnel', 'HomeTeamAbbr',
       'VisitorTeamAbbr', 'Week', 'Stadium', 'Location', 'StadiumType', 'Turf',
       'GameWeather', 'Temperature', 'Humidity', 'WindSpeed', 'WindDirection'],
      dtype='object')

In [239]:
bdb_pass_2018.columns

Index(['gameId', 'quarter', 'down', 'yardsToGo', 'possessionTeam',
       'yardlineSide', 'yardlineNumber', 'offenseFormation', 'personnelO',
       'defendersInTheBox', 'numberOfPassRushers', 'personnelD',
       'preSnapVisitorScore', 'preSnapHomeScore', 'gameClock',
       'absoluteYardlineNumber', 'offensePlayResult', 'playResult'],
      dtype='object')

In [228]:
# Do all transformations on a toy dataset first

toy = bdb_rush_2018.head(1000)
## Create DefensiveTeam variable
toy['DefTeam'] = np.where(toy['PossessionTeam'] == toy['HomeTeamAbbr'], toy['VisitorTeamAbbr'], toy['HomeTeamAbbr'])

## change GameClock to integer of minutes
min_left_qtr = pd.to_datetime(toy['GameClock'].str[:5])
toy['Time Left in Quarter'] = min_left_qtr.dt.hour + (min_left_qtr.dt.minute/60)

## Yards to go for first = Distance
## Yards to go for TD = 100-YardLine
toy['Distance for TD'] = 100 - toy['YardLine']

## Current score in the game
## Difference between PossessionTeam and DefTeam
## First create "PosTeamScore" and "DefTeamScore", then take the difference
toy['PosTeamScore'] = np.where(toy['PossessionTeam'] == toy['HomeTeamAbbr'], toy['HomeScoreBeforePlay'], toy['VisitorScoreBeforePlay'])
toy['DefTeamScore'] = np.where(toy['DefTeam'] == toy['HomeTeamAbbr'], toy['HomeScoreBeforePlay'], toy['VisitorScoreBeforePlay'])
toy['ScoreDifference'] = toy['PosTeamScore'] - toy['DefTeamScore']

## create indicator if team is in Redzone (<= 20 yds to go from goal)
toy['Redzone'] = np.where(toy['Distance for TD'] <= 20, 1, 0)

## create indicator if time left in HALF is <= 2min
toy['Under2Min'] = np.where(((toy['Time Left in Quarter']<=2.0) & (toy['Quarter']==2)|(toy['Quarter']==4)), 1, 0)
## need to include timeout data, match on old_game_id...do it later
toy

,GameId,YardLine,Quarter,GameClock,PossessionTeam,Down,Distance,FieldPosition,HomeScoreBeforePlay,VisitorScoreBeforePlay,OffenseFormation,OffensePersonnel,DefendersInTheBox,DefensePersonnel,HomeTeamAbbr,VisitorTeamAbbr,Week,Stadium,Location,StadiumType,Turf,GameWeather,Temperature,Humidity,WindSpeed,WindDirection,DefTeam,Time Left in Quarter,Distance for TD,PosTeamScore,DefTeamScore,ScoreDifference,Redzone,Under2Min
0,2018090600,30,1,14:22:00,ATL,2,5,ATL,0,0,I_FORM,"2 RB, 1 TE, 2 WR",7.0,"4 DL, 2 LB, 5 DB",PHI,ATL,1,Lincoln Financial Field,"Philadelphia, Pa.",Outdoor,Grass,Cloudy,81.0,71.0,8.0,NNW,PHI,14.366667,70,0,0,0,0,0
1,2018090600,41,1,13:46:00,ATL,1,10,ATL,0,0,SINGLEBACK,"1 RB, 1 TE, 3 WR",7.0,"4 DL, 2 LB, 5 DB",PHI,ATL,1,Lincoln Financial Field,"Philadelphia, Pa.",Outdoor,Grass,Cloudy,81.0,71.0,8.0,NNW,PHI,13.766667,59,0,0,0,0,0
2,2018090600,6,1,12:15:00,ATL,1,6,PHI,0,0,SINGLEBACK,"1 RB, 1 TE, 3 WR",7.0,"4 DL, 2 LB, 5 DB",PHI,ATL,1,Lincoln Financial Field,"Philadelphia, Pa.",Outdoor,Grass,Cloudy,81.0,71.0,8.0,NNW,PHI,12.250000,94,0,0,0,0,0
3,2018090600,1,1,11:41:00,ATL,2,1,PHI,0,0,JUMBO,"2 RB, 3 TE, 0 WR",10.0,"6 DL, 3 LB, 2 DB",PHI,ATL,1,Lincoln Financial Field,"Philadelphia, Pa.",Outdoor,Grass,Cloudy,81.0,71.0,8.0,NNW,PHI,11.683333,99,0,0,0,0,0
4,2018090600,1,1,10:55:00,ATL,4,1,PHI,0,0,JUMBO,"2 RB, 3 TE, 0 WR",11.0,"6 DL, 3 LB, 2 DB",PHI,ATL,1,Lincoln Financial Field,"Philadelphia, Pa.",Outdoor,Grass,Cloudy,81.0,71.0,8.0,NNW,PHI,10.916667,99,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2018091606,38,1,01:42:00,PHI,1,10,PHI,7,0,SHOTGUN,"6 OL, 1 RB, 1 TE, 2 WR",8.0,"4 DL, 3 LB, 4 DB",TB,PHI,2,Raymond James Stadium,"Tampa, FL",Outdoor,Natural Grass,Cloudy,91.0,65.0,9,SSW,TB,1.700000,62,0,7,-7,0,0
996,2018091606,47,1,00:56:00,TB,1,10,PHI,7,0,SHOTGUN,"1 RB, 2 TE, 2 WR",8.0,"4 DL, 3 LB, 4 DB",TB,PHI,2,Raymond James Stadium,"Tampa, FL",Outdoor,Natural Grass,Cloudy,91.0,65.0,9,SSW,PHI,0.933333,53,7,0,7,0,0
997,2018091606,20,2,14:04:00,TB,1,10,TB,7,0,I_FORM,"1 RB, 2 TE, 2 WR",7.0,"4 DL, 3 LB, 4 DB",TB,PHI,2,Raymond James Stadium,"Tampa, FL",Outdoor,Natural Grass,Cloudy,91.0,65.0,9,SSW,PHI,14.066667,80,7,0,7,0,0
998,2018091606,47,2,11:52:00,TB,2,10,TB,7,0,SINGLEBACK,"1 RB, 1 TE, 3 WR",7.0,"4 DL, 2 LB, 5 DB",TB,PHI,2,Raymond James Stadium,"Tampa, FL",Outdoor,Natural Grass,Cloudy,91.0,65.0,9,SSW,PHI,11.866667,53,7,0,7,0,0


# Models

# Preliminary Results